In [ ]:
pip install --upgrade huggingface_hub

In [ ]:
# Authentication
# 이거 토큰이 있는데, 카톡으로...줄까? 깃 env파일에 올리면 좋은데 그 방법을 까먹었어..
from huggingface_hub import login
login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load Model

"""
참고로 모델이 35GB가 넘는 것 같은데 아마 gpu 캐시...? 이런 곳에 저장되는 것 같아서,
뭔가 모델을 따로 저장할 공간이 있다면(로컬이나 구글드라이브) 저장해서 쓰는게 좋을 것 같아!
  => export TRANSFORMERS_CACHE=~/transformers_cache 이런식으로 바꿀 수 있다는데 TRANSFORMERS_CACHE라는 환경변수에 대해 솔직히 잘 모르겠어..ㅠ

그래서 런타임이 중단되면 다시 다운받아야하고, 한 15분 이내로 걸려!
..ㅜㅡㅜ
"""

model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/81.1k [00:00<?, ?B/s]

...

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
def EXAONE_en2ko_translation(input_text):

    prompt = f"""다음 문장을 한국어로 번역해줘.
{input_text}

'다음은 번역 결과입니다.'같은 말 넣지 말고 그냥 번역한 문장만 출력해줘."""

    messages = [
        {"role": "system", "content": "You are a translator that translates from English to Korean."},
        {"role": "user", "content": prompt}
    ]

    # 인풋으로 받은 영어 문장+프롬프트 인코딩
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    # 번역 결과 생성
    output = model.generate(
        input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=128
    )

    # 출력 디코딩
    # translated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    output_text = tokenizer.decode(output[0])
    start_idx = output_text.find("[|assistant|]") + len("[|endofturn|]")
    output_text = output_text[start_idx:].strip()  # 앞 제거

    endofturn_idx = output_text.rfind("[|endofturn|]")
    translated_text = output_text[:endofturn_idx].strip() # 뒤 제거


    return translated_text


In [ ]:
# input_text = "Do you like chicken?"
# translated_text = EXAONE_en2ko_translation(input_text)


file_path = './refined_data/output_en_A2.txt'
file_path = './refined_data/output_en_C2.txt'

# 파일을 열고 2500줄까지 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i >= 1:  # A에서 2500줄, B에서 2500줄씩 받아올거임
            break
        input_text = line.strip()
        translated_text = EXAONE_en2ko_translation(input_text)

# print(translated_text)


FileNotFoundError: [Errno 2] No such file or directory: './refined_data/output_en_A2.txt'